# Интерполяция функции

**Цель работы**: решить задачу интерполяции — нахождения значения функции $y = f(x)$, заданной на отрезке $[a, b]$ парами $(x_i, y_i)$, в тех точках $x\in[a,b]$, для которых пара $(x, y)$ отсутствует, т.е. для промежуточных аргументов.

Для исследования использовать:
* линейную и квадратичную интерполяцию
* многочлен Лагранжа
* многочлен Ньютона


## Линейная интерполяция

Линейная интерполяция _локальна_ (различные коэффициенты между каждым из узлов $(x_{i-1}, y_{i-1})$, $(x_i, y_i)$). Для нахождения значения функции в точке $x$ необходимо:
1. Найти два смежных узла $x_{i-1}$ и $x_i$, между которыми располагается $x$, то есть выполняется условие $x_{i-1} \leq x \leq x_i$;
2. Соединить $x_{i-1}$ и $x_i$ прямой $y = ax + b$, коэффициенты которой определяются как:
$$a = \frac{y_i - y_{i-1}}{x_i - x_{i-1}},\ b = y_{i-1} - a \cdot x_{i-1}$$;
3. Найти значение функции, подставив $x$ в уравнение прямой.

In [1]:
def linear_interp(xy_table, x):
  i = next(i for i, [x_i, y_i] in enumerate(xy_table) if x_i > x)
  x_i, y_i = xy_table[i]
  x_i_prev, y_i_prev = xy_table[i - 1]
  
  a = (y_i - y_i_prev) / (x_i - x_i_prev)
  b = y_i_prev - a*x_i_prev
  
  return a*x + b

## Квадратичная интерполяция

Для нахождения коэффициентов квадратичной интерполяционной функции $y = a_ix^2 + b_ix + c_i$ необходимы значения трех узлов — $x_{i-1}$, $x_i$, $x_{i+1}$:

\begin{cases} a_i x_{i-1}^2 + b_i x_{i-1} + c_i = y_{i-1} \\ a_i x_i^2 + b_i x_i + c_i = y_i \\ a_i x_{i+1}^2 + b_i x_{i+1} + c_i = y_{i+1} \end{cases}

В матричной форме система примет следующий вид:


$$\begin{bmatrix}
  x_{i-1}^2 & x_{i-1} & 1 \\
  x_i^2 & x_i & 1 \\
  x_{i+1}^2 & x_{i+1} & 1
\end{bmatrix} \cdot \begin{bmatrix}
a_i \\ b_i \\ c_i
\end{bmatrix} = \begin{bmatrix}
y_{i-1} \\ y_i \\ y_{i+1}
\end{bmatrix}$$

In [2]:
import numpy as np

def quadratic_interp(xy_table, x):
  i = next(i for i, [x_i, y_i] in enumerate(xy_table) if x_i > x)
  [x1, y1], [x2, y2], [x3, y3] = xy_table[(slice(i-1,i+2) if i < len(xy_table) - 1 else slice(i-3,i))] 
  
  a, b, c = np.linalg.solve(np.array([[x1**2, x1, 1], [x2**2, x2, 1], [x3**2, x3, 1]]), np.array([y1, y2, y3]))
  
  return a*x**2 + b*x + c

## Многочлен Лагранжа

Неизвестные значения функции $f(x) = y$ вычисляются как значения интерполяционного многочлена Лагранжа $Ln(x)$, который принимает данные значения $y_0, ..., y_i$ в наборе точек $x_0, ..., x_i$.

$$Ln(x) = \sum_{i=0}^{n-1} y_i \prod_{j=0,\ j\neq i}^{n-1} \frac{x - x_j}{x_i - x_j}$$

In [3]:
def lagrange_interp(xy_table, x):
  return sum(y_i * np.prod(list((x - x_j) / (x_i - x_j) for j, [x_j, _] in enumerate(xy_table) if i != j))
             for i, [x_i, y_i] in enumerate(xy_table))

## Интерполяционные формулы Ньютона

Если узлы интерполяции равноотстоящие и упорядочены по величине, то есть $x_{i+1} - x_i = h = const$, то интерполяционный многочлен может быть записан в форме Ньютона.

### Прямая формула (для интерполирования вперед)

Для вычисления значений функции в точках левой половины отрезка обычно используют следующую формулу:

$$Nn(x) = y_i + t\Delta y_{i-1} + \frac{t(t-1)}{2!}\Delta^2 y_i + ... + \frac{\prod_{j=0}^{n-1} (t-j)}{n!}\Delta^n y_i,\ t = \frac{x - x_i}{h}$$

Где $\Delta y_i = y_{i+1} - y_i$ (конечная разность первого порядка), $\Delta^n y_i = \Delta^{n-1} y_{i+1} - \Delta^{n-1} y_i$ (конечная разность $n$-ого порядка).

In [4]:
from math import factorial

def divided_diff(ys, n, i):
  return (ys[i + 1] - ys[i]) if n == 0 else (divided_diff(ys, n - 1, i + 1) - divided_diff(ys, n - 1, i))

def newton_forward_interp(xy_table, x):
  xs, ys = np.transpose(xy_table).tolist()
  i = next(i - 1 for i, x_i in enumerate(xs) if x_i > x)
  x_i = xs[i]
  h = xs[1] - xs[0]
  t = (x - x_i) / h

  return ys[i] + sum(np.prod(list(t - k for k in range(j))) / factorial(j - 1) * divided_diff(ys, j - 1, i)
             for j in range(1, len(xs) - 1 - i))

## Загрузка входных данных из файла

_Вариант 10_

In [5]:
import numpy as np

filename = 'input'
xy_table = np.sort(np.loadtxt(filename), axis=0).tolist()

print(f'Результат линейной интерполяции для x = 0.622: {linear_interp(xy_table, 0.622)}')
print(f'Результат квадратичной интерполяции для x = 0.622: {quadratic_interp(xy_table, 0.622)}')
print(f'Результат интерполяции многочленом Лагранжа для x = 0.622: {lagrange_interp(xy_table, 0.622)}')

print(f'Результат интерполяции прямой формулой Ньютона для x = 0.622: {newton_forward_interp(xy_table, 0.622)}')

Результат линейной интерполяции для x = 0.622: 0.5524625
Результат квадратичной интерполяции для x = 0.622: 0.5524480769230771
Результат интерполяции многочленом Лагранжа для x = 0.622: 0.5524807736075374
Результат интерполяции прямой формулой Ньютона для x = 0.622: 0.5537


In [6]:
xy_table

[[0.593, 0.532],
 [0.598, 0.5356],
 [0.605, 0.5406],
 [0.613, 0.5462],
 [0.619, 0.5504],
 [0.627, 0.5559],
 [0.632, 0.5594]]